In [ ]:
# 安裝 gensim
!pip install gensim

In [1]:
# 匯入套件
from gensim.models import Word2Vec
import jieba

In [2]:
# 句子 (也可以是一篇文章)
sentences = [
    "我喜歡閱讀書籍",
    "我也喜歡使用電腦來學習新的知識",
    "閱讀可以開闊我們的視野",
    "電腦是現代學習的重要工具"
]

# 使用 jieba 進行斷詞
sentences = [jieba.lcut(sentence) for sentence in sentences]

# 設定參數
sg = 0 # sg=1 -> skip-gram, sg=0 -> cbow

# 向前看幾個字或向後看幾個字
window_size = 2

# 向量維度
vector_size = 100

# 訓練幾回
epochs = 20

# 最少多少個字才會被使用
min_count = 1

# seed
seed = 42

# 建立 Word2Vec 模型
model = Word2Vec(
    sentences, 
    vector_size=vector_size, 
    window=window_size, 
    sg=sg, 
    min_count=1,
    seed=seed,
    epochs=epochs)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\darren\AppData\Local\Temp\jieba.cache
Loading model cost 0.663 seconds.
Prefix dict has been built successfully.


In [3]:
# 取得 "閱讀" 這個詞的詞向量
vector = model.wv['閱讀']

# 輸出 "閱讀" 的詞向量
print(vector)

[ 6.5954071e-03 -5.9868670e-03  6.1242823e-03 -9.8507879e-03
  5.9283017e-03  5.7808924e-03  5.5459896e-03  3.3354512e-03
 -6.2992523e-04  4.0789861e-03 -4.4583189e-03  5.6846226e-03
  1.1682488e-03 -7.4988086e-04  1.7718112e-04  1.4477749e-03
 -9.2442632e-03 -7.2804657e-03 -5.0738980e-03 -7.6958109e-03
 -1.2995761e-03  3.3611779e-03  3.0769682e-03 -5.1966432e-04
  7.2063389e-03  1.2711901e-03 -8.4013008e-03  5.2771349e-03
  1.4740955e-03  2.6925886e-03  1.3152183e-03  1.0411168e-03
 -8.2053021e-03  1.2484069e-03  5.9369495e-03 -3.9278842e-03
  2.1179605e-03 -9.4857570e-03 -3.0904261e-03 -1.2018590e-03
  9.6746739e-03 -5.6946776e-03 -4.4047888e-03 -1.8955042e-03
  9.8145949e-03  7.0748185e-03 -9.2347665e-03 -5.2571776e-03
  6.5050884e-03 -8.8492902e-03  7.1549197e-03 -4.4357688e-03
  8.2967877e-03 -4.0846397e-03 -1.2842315e-03  3.2024672e-03
 -7.4801035e-03  1.1386370e-03  8.3789768e-05  5.7440619e-03
  9.9593448e-03  3.3219403e-03 -1.8971625e-03 -1.7824204e-03
 -1.6002868e-03  6.25934

In [4]:
# 儲存模型
model.save('word2vec.model')

In [5]:
# 讀取模型
loaded_model = Word2Vec.load("word2vec.model")

In [6]:
# 尋找相近的字詞
loaded_model.wv.most_similar('閱讀', topn=10)

[('是', 0.2088485062122345),
 ('重要', 0.17286993563175201),
 ('可以', 0.15483269095420837),
 ('現代學習', 0.13875289261341095),
 ('的', 0.09962724894285202),
 ('學習', 0.0726875513792038),
 ('我們', 0.029159801080822945),
 ('電腦', 0.0044628651812672615),
 ('喜歡', 0.00015831770724616945),
 ('也', -0.0144733851775527)]

In [7]:
# 計算相近度
loaded_model.wv.similarity('書籍', '知識')

0.10312064